# Part 7 : Working with Runs

Universal code that we will use for the entire notebook.

In [2]:
# required libraries
from openai import OpenAI

# libraries needed for streaming output
from typing_extensions import override
from openai import AssistantEventHandler

# additional libraries
import time


In [3]:
# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [4]:

# Create an assistant.
assistant = client.beta.assistants.create(
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    name="Run Tester Assistant",
    metadata={
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_JeLrnHWuAPfGEmIwMMJFuHwg', created_at=1715656664, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4o', name='Run Tester Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Run Tester Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [5]:
# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread)


Thread(id='thread_SEn5nht5DTkZuaZNYr9jxpLO', created_at=1715656664, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [6]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Tell me what a penguin is in 100 words or less.",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_zCqpSKppaMxANFlRKKTH8aAK', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')], created_at=1715656664, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SEn5nht5DTkZuaZNYr9jxpLO')


msg_zCqpSKppaMxANFlRKKTH8aAK
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]
Tell me what a penguin is in 100 words or less.
user


## Creating Runs
Runs are the engine that makes things happen with the LLM. 

In [7]:

run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

print(run)


Run(id='run_beE7XeciSxAUwdnCHeiURwnu', assistant_id='asst_JeLrnHWuAPfGEmIwMMJFuHwg', cancelled_at=None, completed_at=None, created_at=1715656665, expires_at=1715657265, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_SEn5nht5DTkZuaZNYr9jxpLO', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


### Getting the Run Status
You can't just keep looking at the run result to get the current status

In [8]:
print(run)
print("\n")
print(run.status)

Run(id='run_beE7XeciSxAUwdnCHeiURwnu', assistant_id='asst_JeLrnHWuAPfGEmIwMMJFuHwg', cancelled_at=None, completed_at=None, created_at=1715656665, expires_at=1715657265, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_SEn5nht5DTkZuaZNYr9jxpLO', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


queued


You have to get the run status by retrieving the run manually

In [9]:
runstatus = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

print(runstatus)
print("\n")
print(runstatus.id)
print(runstatus.last_error)
print(runstatus.status)

Run(id='run_beE7XeciSxAUwdnCHeiURwnu', assistant_id='asst_JeLrnHWuAPfGEmIwMMJFuHwg', cancelled_at=None, completed_at=None, created_at=1715656665, expires_at=1715657265, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_SEn5nht5DTkZuaZNYr9jxpLO', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


run_beE7XeciSxAUwdnCHeiURwnu
None
queued


Or you can poll the status manually with a loop

In [10]:
# sleep for 10 seconds to let the run finish
# in case you want to run the entire notebook
time.sleep(10)

In [11]:
import openai
import time

another_run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

# Retrieve initial run status
run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Poll the status while it's still queued or in progress
while run_status.status in ["queued", "in_progress"]:
    # Log current status
    print(f"Run status: {run_status.status}")
    
    # Wait for 1 second
    time.sleep(1)
    
    # Retrieve the status again
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Optionally, print the final status or any other detail
print("Final status:", run_status.status)


Run status: queued
Run status: in_progress
Final status: completed


The best option is to save yourself some code and just use the "create_and_poll" method to do the same thing. 

When interacting with the API some actions such as starting a Run and adding files to vector stores are asynchronous and take time to complete. The SDK includes helper functions which will poll the status until it reaches a terminal state and then return the resulting object. If an API method results in an action which could benefit from polling there will be a corresponding version of the method ending in '_and_poll'.

In [12]:
auto_run_and_poll = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

print(auto_run_and_poll)
print("\n")
print(auto_run_and_poll.id)
print(auto_run_and_poll.status)

Run(id='run_ljFutZuysg9OuCf6Hg0yeTsK', assistant_id='asst_JeLrnHWuAPfGEmIwMMJFuHwg', cancelled_at=None, completed_at=1715656680, created_at=1715656678, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715656678, status='completed', thread_id='thread_SEn5nht5DTkZuaZNYr9jxpLO', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=115, prompt_tokens=252, total_tokens=367), temperature=1.0, top_p=1.0, tool_resources={})


run_ljFutZuysg9OuCf6Hg0yeTsK
completed


### Creating a Simple Streaming Run
Instead of having to wait for the entire run to finish before getting a result, it is best to stream output to the user for interactive sessions.

In [13]:
# Create a new run with the stream option set to True
# Attempt to create a new run
stream_run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    stream=True
)

# Continue processing the new run
for event in stream_run:
    if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
        print(event.data.id)
        print(event.data.status)
    else:
        print(f"Event ID: {event.data.id} does not have a status attribute.")
        print(event.data.delta)
    print("---------------\n")


run_u8nMMa9OSnMCerj99kMf9Txa
queued
---------------

run_u8nMMa9OSnMCerj99kMf9Txa
queued
---------------

run_u8nMMa9OSnMCerj99kMf9Txa
in_progress
---------------

step_Oc7dUHOQOXAbAJb0YDq48ysB
in_progress
---------------

step_Oc7dUHOQOXAbAJb0YDq48ysB
in_progress
---------------

msg_iQktZbU2kBRvwLFGFjDhMi8v
in_progress
---------------

msg_iQktZbU2kBRvwLFGFjDhMi8v
in_progress
---------------

Event ID: msg_iQktZbU2kBRvwLFGFjDhMi8v does not have a status attribute.
MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=[], value='A'))], role=None)
---------------

Event ID: msg_iQktZbU2kBRvwLFGFjDhMi8v does not have a status attribute.
MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value=' peng'))], role=None)
---------------

Event ID: msg_iQktZbU2kBRvwLFGFjDhMi8v does not have a status attribute.
MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value='uin'))], role=

### Dealing with Thread Locks

#### Notification of Failed Run
You can't have multiple runs on a thread. To deal with situations where this might occur, you can check to see if a run is already taking place and get feedback on your failed run.

In [14]:
# create a run to interfere with the other run
interference_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

# Function to check if there is an active run
def check_active_run(thread_id):
    try:
        # Fetch runs for the thread and check their status
        active_runs = client.beta.threads.runs.list(thread_id=thread_id)
        for run in active_runs.data:
            if run.status in ["in_progress", "queued"]:
                return run.id
        return None
    except OpenAIError as e:
        print(f"Failed to check runs: {str(e)}")
        return None

# Get the active run ID, if any
active_run_id = check_active_run(thread.id)
if active_run_id:
    print(f"Thread already has an active run: {active_run_id}. Please wait until it completes.")
else:
    try:
        # Attempt to create a new run if no active run exists
        stream_run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
            stream=True
        )

        # Continue processing the new run
        for event in stream_run:
            if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
                print(event.data.id)
                print(event.data.status)
            else:
                print(f"Event ID: {event.data.id} does not have a status attribute.")
                print(event.data.delta)  # Ensure that event.data.delta exists or handle it similarly
            print("---------------\n")

    except OpenAIError as e:  # Handle generic OpenAIError if BadRequestError is not explicitly available
        print(f"Error occurred: {e}")
        if "already has an active run" in str(e):
            print("A run is already active on this thread. Please wait until it completes.")

Thread already has an active run: run_wnv5D9q5KEHsMHGjMM1mHbVp. Please wait until it completes.


In [15]:
# sleep for 10 seconds to let the run finish
# in case you want to run the entire notebook
time.sleep(10)

#### Waiting Your Turn
A better strategy is to check to see if there is already a run going on and just poll until the run is complete then do your run afterward

In [16]:

# create a run to interfere with the other run
interference_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )


# Wait until there is no active run
while check_active_run(thread.id):
    print("Waiting for the existing run to complete...")
    time.sleep(1)  # Wait for 10 seconds before checking again

# Once no active runs are detected, proceed to create a new run
try:
    print("Creating a new run...")
    stream_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

    # Continue processing the new run
    for event in stream_run:
        if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
            print(event.data.id)
            print(event.data.status)
        else:
            print(f"Event ID: {event.data.id} does not have a status attribute.")
            print(event.data.delta)  # Ensure that event.data.delta exists or handle it similarly
        print("---------------\n")

except OpenAIError as e:
    print(f"Error occurred: {e}")


Waiting for the existing run to complete...
Waiting for the existing run to complete...
Creating a new run...
run_lwQZzVSQtS8zaJYn0gOA0cAD
queued
---------------

run_lwQZzVSQtS8zaJYn0gOA0cAD
queued
---------------

run_lwQZzVSQtS8zaJYn0gOA0cAD
in_progress
---------------

step_egi36bO5oAyn5btRxMC5DHRK
in_progress
---------------

step_egi36bO5oAyn5btRxMC5DHRK
in_progress
---------------

msg_5ou7E0TAON99SW0nwwrllUPf
in_progress
---------------

msg_5ou7E0TAON99SW0nwwrllUPf
in_progress
---------------

Event ID: msg_5ou7E0TAON99SW0nwwrllUPf does not have a status attribute.
MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=[], value='A'))], role=None)
---------------

Event ID: msg_5ou7E0TAON99SW0nwwrllUPf does not have a status attribute.
MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value=' peng'))], role=None)
---------------

Event ID: msg_5ou7E0TAON99SW0nwwrllUPf does not have a status attribute.
Mes

# Part 8: More Working with Runs
## Getting the Output from the Model

### Output After Polling
If we do polling, we can get the output by fetching the last Assistant message from the run when it is done. The last message should be the Assistant's answer to our query. 

In [17]:
# Create a single run 
fresh_run = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

# Retrieve messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Get the latest assistant message 
latest_assistant_message = None
for message in messages.data:
    if message.role == 'assistant' and message.run_id == fresh_run.id:
        latest_assistant_message = message
        break

# Print the latest response
if latest_assistant_message:
    print("Output:\n" + latest_assistant_message.content[0].text.value) 
else:
    print("No assistant message found for the run.")

Output:
A penguin is a flightless bird native to the Southern Hemisphere, notably Antarctica. Recognizable by their black and white plumage and upright posture, penguins are excellent swimmers but lack the ability to fly. They are well adapted to cold environments, with a layer of insulating blubber and tightly packed feathers. Penguins primarily feed on fish, squid, and krill. These social birds often live in large colonies, exhibiting unique behaviors such as synchronized swimming and cooperative parenting, where both parents share the responsibility of raising their young.


### Output While Streaming
Conversly, when streaming, we want to show the output right away. We do this using events from the stream to show the output as it is produced. 

In [18]:
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
# Normally, you would define this class at the top of your script or in a separate file.
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant text > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant tool > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [19]:

# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant text > A penguin is a flightless bird native to the Southern Hemisphere, particularly Antarctica. Known for their black and white plumage and upright stance, penguins are adept swimmers with flipper-like wings but cannot fly. They are well-adapted to cold climates, having a thick layer of insulating blubber and dense feathers. Penguins primarily feed on fish, squid, and krill. These social birds often form large colonies for breeding and protection, exhibiting synchronized swimming and cooperative parenting behaviors where both parents care for their young.

## Modifying Assistants and Threads with Runs
You can make changes in real-time to your Runs that override or modify existing setting on Assistants and Threads.

### Runs and Assistants
Let's see how we can make changes to the Assistant settings for our Runs.

In [21]:
# Show the Assistant instructions
print("Assistant Instructions: " + assistant.instructions)
print("\n")

# Create a run to modify Assistant instructions
additional_instruction_run = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
    model="gpt-4-turbo",
    additional_instructions="That speaks like a pirate.",
)

print("Run Instructions: " + additional_instruction_run.instructions)
print("\n")

# Retrieve messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Get the latest assistant message 
latest_assistant_message = None
for message in messages.data:
    if message.role == 'assistant' and message.run_id == additional_instruction_run.id:
        latest_assistant_message = message
        break

# Print the latest response
if latest_assistant_message:
    print("Output:\n" + latest_assistant_message.content[0].text.value.strip()) 
else:
    print("No assistant message found for the run.")

# add some space
print("\n")


# print the original assistant instructions
print("Assistant Instructions: " + assistant.instructions)

Assistant Instructions: You are a helpful assistant.


Run Instructions: You are a helpful assistant. That speaks like a pirate.


Output:
A penguin be a flightless bird ye’ll find mainly in the chilly waters and lands of the Southern Hemisphere, especially around Antarctica. They be dressed smartly, like gentlemen of the sea, in their snug black and white feathers, which help keep out the bitter cold. Penguins are sturdy on land but true masters in the water, using their flippers to dart through the sea with ease. They feast on a hearty diet of fish, squid, and krill. Social creatures they are, livin' in large colonies and sharin' duties like raisin' their chicks and hunting. Shiver me timbers, they’re a fascinating bunch!


Assistant Instructions: You are a helpful assistant.


## Controlling Output
We can detemine the temperature and top_p for our runs and override the Assistant settings.

In [23]:
# show the Assistant temperature
print(assistant.instructions)
print(assistant.temperature)

# Stream the response with the new temperature
# Set the temperature to 0.5 to get more conservative responses
# Set the top_p to 0.5 to get more conservative responses
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  additional_messages=[
    {
      "role": "user",
      "content": "Write a story about a magical frog in a forest in 100 words less."
    }
  ],
  temperature=0,
  top_p=0.1,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

print("\n")

# Now we change the temperature with our run
# Set the temperature to 1.2 to get more creative responses
# Set the top_p to 1 to get more creative responses
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  additional_messages=[
    {
      "role": "user",
      "content": "Write a story about a magical frog in a forest in 100 words less."
    }
  ],
  temperature=1.2,
  top_p=1,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

You are a helpful assistant.
1.0

assistant text > In a mystical forest, a magical frog named Zephyr lived by a crystal-clear pond. Zephyr had the power to heal with a touch. One day, a young girl named Lily found Zephyr, her eyes filled with worry for her sick mother. Zephyr gently hopped onto Lily's hand, and a warm, golden light enveloped them. With a soft croak, Zephyr transferred his healing magic to Lily. She hurried home and touched her mother, who instantly regained her health. Grateful, Lily and her mother often visited Zephyr, bringing him treats and sharing stories, forever cherishing the magical frog who saved their family.


assistant text > In a hidden forest grove, a magical frog named Finley lived by a glittering stream. Finley possessed the power to grant courage. One day, a timid rabbit named Hazel approached him, scared of a looming storm. Finley croaked softly, and a shimmering glow surrounded Hazel, filling her with bravery. As the storm approached, Hazel stood tal